In [ ]:
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

In [3]:
import torch
import gc
import os
# Set PYTORCH_CUDA_ALLOC_CONF environment variable
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "0:15360"
import cv2
from ultralytics import YOLO
#import cv2
#import glob
#gc.collect()
#torch.cuda.empty_cache()
import numpy as np
from skimage.transform import resize
from ultralytics.yolo.utils.plotting import Annotator


# Load a model
#model = YOLO('yolov8n-seg.pt')  # load an official models
model = YOLO('runs/segment/train4/weights/best.pt')  # load a custom model

# Predict with the model
results = model('D:\\815_CowDataChecking\\uom\\2023-04-04\\09\\20230404-092500-092959.mp4',save=False,retina_masks=False,show=True,batch=1,imgsz=640,stream=False)  # predict on an image


for result in results:
    #print(result.boxes)
    vid_path = result.path
    filename = vid_path.split("\\")[-1].replace(".mkv","")
    
    
    boxes = result.boxes.cpu().numpy()
    ori_img = result.orig_img
    annotator = Annotator(ori_img)
    box_count = 0
    cow_position = 1
    
    h,w = result.orig_shape
    count = 1
    b_boxes = []
    masks = []
    ids = []
    has_cattle = False
    if  result.masks != None:
        
        #result_masks = result.masks.cpu().numpy().masks.astype(bool)
        for m in result.masks.cpu().numpy().masks.astype(bool):
        #for i in range(1,len(result_masks)+1):
        #for m in result.masks.masks.astype(bool):
            xyxy = boxes[box_count].xyxy[0]
        #   m = result_masks[-1]
            #print(xyxy)
            #print(m.shape)
            box_count += 1
            x1= int(xyxy[0])
            y1= int(xyxy[1])
            x2= int(xyxy[2])
            y2= int(xyxy[3])

          
            #print("not skipped")
            #new_results.append(result)
            box_left = x1
            box_top = y1
            box_w = x2 - x1
            box_h = y2 - y1

            new = np.zeros_like(ori_img, dtype=np.uint8)
            new[m] = ori_img[m]
            #masks.append(m)
            
           
            crop = new[y1:y2, x1:x2]
           

           
    #frame = annotator.result() 
    frame = cv2.resize(ori_img, (1080,1080), interpolation = cv2.INTER_AREA)
    if has_cattle:
        #ori_img = cv2.resize(ori_img,(1088,1088))
        for i in range(len(b_boxes)):
            box = b_boxes[i]
            #mask = masks[i]
            prev_id = 5
            draw_bounding_box(ori_img,(box[0],box[1],box[2],box[3]),str(prev_id))
            #ori_img = overlay(ori_img,mask,(0,0,255),0.3)
        frame = cv2.resize(ori_img, (1080,1080), interpolation = cv2.INTER_AREA)
        cap.write(frame)
    
    
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cv2.destroyAllWindows()



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/5984) D:\815_CowDataChecking\uom\2023-04-04\09\20230404-092500-092959.mp4: 640x384 (no detections), 20.1ms
video 1/1 (2/5984) D:\815_CowDataChecking\uom\2023-04-04\09\20230404-092500-092959.mp4: 640x384 (no detections), 34.0ms
video 1/1 (3/5984) D:\815_CowDataChecking\uom\2023-04-04\09\20230404-092500-092959.mp4: 640x384 (no detections), 35.0ms
video 1/1 (4/5984) D:\815_CowDataChecking\uom\2023-04-04\09\20230404-092500-092959.mp4: 640x384 (no d

KeyboardInterrupt: 

In [2]:
cv2.destroyAllWindows()

In [1]:
from pathlib import Path
import cv2
import numpy as np
from datetime import datetime
import os
import time

X1=230 #same as NEW_BLACK_X1
X2=410 #same as NEW_BLACK_X2 # incase of x2 out of bound
Y1=94
Y2=590
SIZE =224
Y1_NEW=125 #135  #decrease here to extend, increase to shrink 
Y2_NEW=490  #530  # redyce here to extend , increase to do vice casa 460 previous
FRAME = 1

default=640

BATCH = 100
BATCH_COUNT = 1
PREV_BATCH = 0
LAST_SEEN = time.time()
FIRST_SEEN = True
demo_img_save_path = []

prevId_record =[]

#end

#region Cattle Tracking
STORED_IDS= []
STORED_MID_Y = []
STORED_MID_Y1 = []
STORED_MID_Y2 = []
STORED_MISS = []
PREVIOUS_ID = [] # keep the record of last seen ids and position
PREVIOUS_Y1 = [] 
PREVIOUS_Y2 = [] 
PREVIOUS_LOCAL_IDS = []
CATTLE_LOCAL_ID= 1
IS_FIRST_CATTLE = True

###### load dataset and model


def check_withinROI_NEW(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    if(x1<int(X1*(w/default)) or x2>int(X2*(w/default)) or y1<int(Y1_NEW*(h/default)) or y2>int(Y2_NEW*(h/default)) or x1>=int(X2*(w/default))):
        return False
    if(y2 - y1>1400 or y2-y1<400): #1400 to 700 Before
        return False
    return True  

def check_withinROI_Resize(x1,y1,x2,y2,h,w):
    #print(x1, '  ',y1, '  ',x2, '  ',y2, '  ',h, '  ',w)
    resize_x1=850#*(w/2992)
    resize_x2=1050#*(w/2992)
    resize_y1=Y1_NEW#Y1_NEW*(w/2992)
    resize_y2=Y1_NEW#Y2_NEW*(w/2992)
    #print(resize_x1, '  ',resize_y1, '  ',resize_x2, '  ',resize_y2)
    if(x1<int(resize_x1) or x2>int(resize_x2) or x1>=int(resize_x2)):
        return False
    if(y2 - y1>1400 or y2-y1<200): #1400 to 700 Before
        return False
    return True  


################################################################################    

def Is_Duplicate_Id(y1,y2,id):
    global PREVIOUS_ID
    global PREVIOUS_Y1
    global PREVIOUS_Y2
    global PREVIOUS_LOCAL_IDS
    global CATTLE_LOCAL_ID
    
    try: 
        index = PREVIOUS_ID.index(id)
        #print('I reached here')
        if(PREVIOUS_Y1[index]+351<=y1 and PREVIOUS_Y2[index]+371<y2): #duplicate from bottom
           
            PREVIOUS_ID.append(CATTLE_LOCAL_ID)
            PREVIOUS_Y1.append(y1)
            PREVIOUS_Y2.append(y2)
            #print('New Cattle Id')
            return CATTLE_LOCAL_ID
        
        else:

            PREVIOUS_Y1[index]=y1 #duplicate is solved or no duplicate and just need for last y 
            PREVIOUS_Y2[index]=y2
            return PREVIOUS_ID[index]
    except:
        CATTLE_LOCAL_ID += 1
        #print('except')
        PREVIOUS_ID.append(CATTLE_LOCAL_ID)
        PREVIOUS_Y1.append(y1)
        PREVIOUS_Y2.append(y2)
        return id
        
#def Take_Prev_Label(y2,id,cow_srno):
def Take_Prev_Label(y,h,id,cow_srno):
    global STORED_IDS
    global STORED_MID_Y
    global STORED_MID_Y1
    global STORED_MID_Y2
    global STORED_MISS
    global LAST_SEEN_IDS
    global LAST_SEEN_ID_CENTROIDS
    global CATTLE_LOCAL_ID
    global IS_FIRST_CATTLE 
    y1 , y2 = y , y+h
    
    if IS_FIRST_CATTLE:
        IS_FIRST_CATTLE = False
        id = CATTLE_LOCAL_ID
    #mid_y = y2
    mid_y = int(2*y + h)/2
    IS_NEW = True
    last_id = 999
    last_y1 = 0
    last_y2 = 0
    if(len(STORED_IDS)>0): 
        last_id = STORED_IDS[len(STORED_IDS)-1]
        last_y1 = STORED_MID_Y1[len(STORED_MID_Y1)-1]
        last_y2 = STORED_MID_Y2[len(STORED_MID_Y2)-1]
        MISSED_LEN = len(STORED_MISS)
        #if(IS_NEW):
        
        #    MISSED_LEN -=1
        removed = 0
        for i in range(MISSED_LEN):
            #print(i, ' missed index checking' )
            missed = STORED_MISS[i-removed]
            #print('checking ',i-removed, 'to remove')
            if((missed>100 and len(STORED_MISS)>0) or int(last_id)-1>int(STORED_IDS[i-removed])): #if missed 35 frames
    
                del STORED_MISS[i-removed]  
                del STORED_MID_Y[i-removed]
                del STORED_MID_Y1[i-removed]
                del STORED_MID_Y2[i-removed]
                del STORED_IDS[i-removed]
                removed+=1
                #print('removed')
                
    #clear misses
   
    
    threshold_1 = 250 #300
    threshold_2 = 300  #230
    Distance = 2000
     
    if mid_y <= 1300 or mid_y >= 700:
        threshold_1 = 320 #350
        threshold_2 = 370 #280
    for i in range(1,len(STORED_MID_Y)+1):
        #print(STORED_IDS[-i-1],STORED_MID_Y[-i-1],' ',i)
        
        
        #if(STORED_MID_Y[-i]+threshold_2>=mid_y and STORED_MID_Y[-i]-threshold_1<=mid_y): # and IS_NEW): #previous 150 #200
        if(STORED_MID_Y1[-i]-threshold_1<=y1 and STORED_MID_Y1[-i]+threshold_1-50>=y1) or (STORED_MID_Y2[-i]-threshold_2<=y2 and STORED_MID_Y2[-i]+threshold_2-50>=y2): # and IS_NEW): #previous 150 #200
            if(IS_NEW):
                #print('mid_y ',mid_y,'existing y ',STORED_MID_Y[-i])
                #print('all mid_y ',STORED_MID_Y) 
                #print("Old")
                #print("STORED_MID_Y1",STORED_MID_Y1[-1], " and STORED_MID_Y2", STORED_MID_Y2)
                #print("Y!",y1, " and Y@", y2)
                
                Distance = abs(STORED_MID_Y1[-i] - y1)
                if(abs(STORED_MID_Y2[-i] - y2)<Distance):
                    Distance = abs(STORED_MID_Y2[-i] - y2)
                IS_NEW = False
                STORED_MID_Y1[-i] = y1
                STORED_MID_Y2[-i] = y2
                
                STORED_MISS[-i]=1
                id= STORED_IDS[-i]
                #print(Distance)
                #print(id)
                
            #try:
            #    exist_index = LAST_SEEN_IDS.index(id)
            #    if(LAST_SEEN_ID_CENTROIDS[exist_index]+200>y): # showing old id
            #        LAST_SEEN_ID_CENTROIDS[exist_index] = y
            #except:
            #print('corrected id :',STORED_IDS[-i])
            elif Distance >60:
                STORED_MISS[-i]+=1
            else:
                STORED_MISS[-i]= 15 #reset count to 2 when not moving
        elif(STORED_MID_Y1[-i]<=y1 and STORED_MID_Y2[-i]>=y2):
                STORED_MISS[-i]=5
        else:
            STORED_MISS[-i]+=1    
            
    if(IS_NEW):
        #print('SMY: ',STORED_MID_Y,', new my:',mid_y) 
        #print('new id: ',id)
        updatedID = Is_Duplicate_Id(y1,y2,id)
        #CATTLE_LOCAL_ID+=1
        #updatedID = CATTLE_LOCAL_ID
        if(int(last_id) <int(updatedID) and y1<last_y1-150 and y2<last_y2-150): # duplicate cattle with increased cattleID
            CATTLE_LOCAL_ID-=1
            print("last_id",last_id," updatedID ",updatedID)
            for i in range(len(STORED_MID_Y)-1,0,-1):
                STORED_MISS[i]=15
            return -1
        if(int(last_id)-1>int(updatedID) and last_id!=999):
            return -1
            
    #if(updatedID!=id):
    #    print('orgID: ',id,' updated ID: ',updatedID)
        #id = str(updated_ID)+'_'+str(id)
        
        id=CATTLE_LOCAL_ID
        STORED_IDS.append(id)
        STORED_MID_Y.append(mid_y)
        STORED_MID_Y1.append(y1)
        STORED_MID_Y2.append(y2)
        STORED_MISS.append(1)
    
    #print('returned id :',id)
    
    print(id)
    
    result = []
    result.append(str(id-1))
    
    removed = 0
    for i in range(len(STORED_MID_Y)-1,0,-1):
        if(y1>STORED_MID_Y1[i] and y2>STORED_MID_Y2[i]):
             del STORED_MISS[i-removed]  
             del STORED_MID_Y[i-removed]
             del STORED_MID_Y1[i-removed]
             del STORED_MID_Y2[i-removed]
             del STORED_IDS[i-removed]
             removed+=1
                 
    return result




def CALCULATE_MAX_CATTLE_ID(csv_path):
    print(csv_path, " is csv_path and ")

    data = pd.read_csv(csv_path)

    list_of_csv = [list(row) for row in data.values]

    prev_id_record = [] 
    prev=None

    current_cow = []
    excel_cow_count = []
    boxes = []
    file_locations = []

    for i in range (len(list_of_csv)):
        filtered_id = list_of_csv[i][0]
        actual_id = list_of_csv[i][1]
        file_locations.append(list_of_csv[i][2])
        boxes.append([list_of_csv[i][3],list_of_csv[i][4],list_of_csv[i][5],list_of_csv[i][6]])
        try: 
            index = current_cow.index(actual_id)
            excel_cow_count[index]+=1
        except:
            current_cow.append(actual_id)
            excel_cow_count.append(1)

    maxpos = excel_cow_count.index(max(excel_cow_count))
    cattle_id = current_cow[maxpos]
    return cattle_id,file_locations,boxes


def writeVideo(filePath):
    img_array = []
    size = (302,1080)
    names = ['cow']


    vid_name = os.path.basename(os.path.normpath(filePath))
    vid_path = str(Path(filePath + "/" + vid_name ).with_suffix('.mp4'))
    id,img_locations,*xyxys = CALCULATE_MAX_CATTLE_ID(filePath+"/"+vid_name+".csv")
    out = cv2.VideoWriter(vid_path,cv2.VideoWriter_fourcc(*'mp4v'), 6, size)
    if len(img_locations)<10: 
        return -1

    for ind in range(len(img_locations)):
        img = cv2.imread(img_locations[ind])
        annotator = Annotator(img, line_width=8, example=str(names))
        try:
            annotator.box_label(xyxys[0][ind],str(id), color=(15, 0, 255))
            annotated_img =cv2.resize(annotator.result(),size) 
            out.write(annotated_img)
        except:
            continue
    out.release()
    img_array=[]
    print("done ", vid_name)
    cv2.destroyAllWindows()
    return id
########################################################################

In [ ]:
from ultralytics.yolo.utils.files import increment_path
import torch
import gc
#from pathlib import Path
gc.collect()
torch.cuda.empty_cache()

from ultralytics.yolo.utils.plotting import Annotator

from ultralytics import YOLO

model = YOLO('runs/segment/train4/weights/best.pt')  # load a custom model

# Predict with the model
project = 'D:/Python/SULarbmon/Python/env/yolov8_june/ultralytics/runs/segment/uom'
name = 'uom'
results = model('D:\\815_CowDataChecking\\uom\\2023-04-04\\09\\20230404-092500-092959.mp4',imgsz=1088,save=False,retina_masks=False,show=False,stream=True,device = '0')
save_dir = increment_path(Path(project) / name, exist_ok=True)  # increment run
#save_dir = increment_path(Path(project) / name,mkdir=True)
#(save_dir / 'labels' if False else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
manual_cow_count = 0

#cap.set(4, 480)
cap = cv2.VideoWriter(os.path.join(save_dir,'output.mkv'),cv2.VideoWriter_fourcc(*'mp4v'), 13, (1088,1088))

for result in results:
    print("here")
       
cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
vid_path = result.path
    filename = vid_path.split("\\")[-1].replace(".mkv","")
    
    
    boxes = result.boxes.cpu().numpy()
    ori_img = cv2.resize(result.orig_img, (1088,1088), interpolation = cv2.INTER_AREA)
    annotator = Annotator(ori_img)
    box_count = 0
    cow_position = 1
    
    h,w = result.orig_shape
    if  result.masks != None:
        
        for m in result.masks.cpu().numpy().masks.astype(bool):
        #for m in result.masks.masks.astype(bool):
            xyxy = boxes[box_count].xyxy[0]
            #print(xyxy)
            #print(m.shape)
            box_count += 1
            x1= int(xyxy[0])
            y1= int(xyxy[1])
            x2= int(xyxy[2])
            y2= int(xyxy[3])

            ################## Validate  #####################
            if(check_withinROI_NEW(x1,y1,x2,y2,h,w)==False):
                continue
            #print("not skipped")
            #new_results.append(result)
            box_left = x1
            box_top = y1
            box_w = x2 - x1
            box_h = y2 - y1

            new = np.zeros_like(ori_img, dtype=np.uint8)
            new[m] = ori_img[m]
            
            x1= int(x1 * (1088/2992))
            x2= int(x2 * (1088/2992))
            y1= int(y1 * (1088/2992))
            y2= int(y2 * (1088/2992))

            crop = new[y1:y2, x1:x2]
            ###### LABEL
            #label = predict_hog_svm(crop)
            label = 1
            prev_id = Take_Prev_Label(box_top,box_h,label,cow_position)
          #########################################  




            HAS_COW=True

            if(prev_id==-1): #skip cattle when prev_id // filter id is -1
                if(count==1):
                    has_seen_cattle=False
                    count-=1
                print('skipped')
                continue
            #print(prev_id)
            cow_position+=1 
            BATCH_COUNT = prev_id[0] # skip batch count here  
            #BATCH calculator

            ### annotate ######
            b = xyxy  # get box coordinates in (top, left, bottom, right) format
            #c = box.cls
            annotator.box_label(b, str(prev_id[0]))

            ###################### CREATE dir to save img
            base_path = str(Path(save_dir / prev_id[0]))
            if not os.path.exists(base_path):
                os.makedirs(base_path)

            LAST_SEEN = time.time()

            FRAME+=1
            #save each images for taking max label later
            ######################################

            demo_annotated_img_save_path = Path(base_path+ '/' + f'{filename}_{str(manual_cow_count).zfill(4)}.jpg')
            try:
                cv2.imwrite(str(demo_annotated_img_save_path), crop)
                print(demo_annotated_img_save_path)
            except:
                print('cannot save ',demo_annotated_img_save_path)
            #change cropped size here  #230 to 215 410 to 390

            manual_cow_count += 1

    frame = annotator.result()  
    frame = cv2.resize(frame, (1080,1080), interpolation = cv2.INTER_AREA)
    cap.write(frame)
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break
     